<a href="https://colab.research.google.com/github/alinnman/celestial-navigation/blob/main/colab/vacation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title

################################################
# Staging core library and .CSV files
################################################

from os import chdir
from pathlib import Path
import subprocess

link_prefix = "https://github.com/alinnman/celestial-navigation/raw/refs/heads/main/"
data_prefix = "sample_data/"

for t in ["starfix"]:
    s = t + ".py"
    my_file = Path (s)
    if not my_file.exists():
        print ("Fetching Python library.")
        subprocess.run (["wget", link_prefix + s])
    else:
        print ("Python library exists.")

try:
    chdir (data_prefix)

    for t in ["planets","stars","sun-moon-sd","sun-moon","venus-mars-hp"]:
        s = t + ".csv"
        my_file = Path(s)
        if not my_file.exists():
            print ("Fetching " + s + ".")
            subprocess.run (["wget", link_prefix + data_prefix + s])
        else:
            print (s + " exists.")
finally:
    chdir ("..")

Python library exists.
planets.csv exists.
stars.csv exists.
sun-moon-sd.csv exists.
sun-moon.csv exists.
venus-mars-hp.csv exists.


# Celestial Navigation Workbench

This is a sample web app for celestial navigation for a stationary observer.<br/>

Instruction:

1. The entered parameters in the three cells below were taken for an observation taken by me on vacation this summer.
Leave the parameters unchanged (use as a demo) or edit the cells to handle a different observation.
1. **Press Ctrl+F9 to run the notebook**. <br/>
On a mobile phone or other keyboard-less device use the menu alternative "Runtime --> Run all" <br/>
This will execute a sight reduction and the bottom cell will show the coordinate and a link to a map of the resulting intersections. Failure to perform a sight reduction will produce an error message.

**NOTE**: When running the notebook for the first time you may get a security warning about the code not originating from Google Colab. You can safely ignore this warning and continue.

**NOTE**: The first execution of the notebook can take some time. This is due to Google Colab initialization.

This is part of (and a demo of) a software package, primarily for using on a mobile phone (without need for an internet connection). It is written in Python and also uses Jupyter/Colab features for web browser support. The target platform is the PyDroid 3 app for Android. For more information see [here](https://github.com/alinnman/celestial-navigation/tree/main/README.md).

For a short intro to the workflow and algorithm used see [here](https://github.com/alinnman/celestial-navigation/tree/main/WORKFLOW.md).

Geographical Positions of celestial objects (Declination, SHA, GHA) and parallax information (HP) are initialized from
a machine-readable nautical almanac residing in the "sample_data" data folder.
The data covers the years 2024-2028.
(These values can also be entered by hand if you use the Python scripts).

A Nautical Almanac for 2024 can be found [here](https://github.com/alinnman/celestial-navigation/blob/main/nautical_almanacs/NAmod(A4)_2024.pdf).

A Nautical Almanac for 2025 can be found [here](https://github.com/alinnman/celestial-navigation/blob/main/nautical_almanacs/NAmod(A4)_2025.pdf).

Format for all angles is "DD:MM:SS", "DD:MM" or "DD" (degrees, minutes, seconds) <br/>
The sight reduction can take care of observer elevation (dip of horizon), atmospheric refraction and parallax.<br/>
For more details and description of parameters see [here](https://github.com/alinnman/celestial-navigation/blob/main/README.md#parameters).

# DRP Parameters

In [2]:
# @title
# Importing Python libraries
from time import time
from starfix import Sight, SightCollection, get_representation, get_google_map_string,\
                    IntersectError, LatLonGeodetic, parse_angle_string,\
                    Circle

In [3]:
# ESTIMATED POSITION / DRP

DRP_LAT       = "55"  # @param {"type":"string"}
DRP_LON       = "18"  # @param {"type":"string"}
DIAGNOSTICS   = False
# Diagnostics don't seem to work in current version of Google Colab

drpLat = parse_angle_string (DRP_LAT)
drpLon = parse_angle_string (DRP_LON)


THE_POS = LatLonGeodetic (drpLat, drpLon)

# SIGHT # 1

In [4]:
# SIGHT NR 1.

USE_1            = True                        # @param {"type":"boolean"}
OBJECT_NAME_1    = "Sun"                       # @param {"type":"string"}
SET_TIME_1       = "2024-06-29 08:21:00+00:00" # @param {"type":"string"}
LIMB_CORR_1      = 0                           # @param [-1,0,1]
INDEX_ERROR_1    = 0                           # @param {"type":"number"}
ARTIFICIAL_HOR_1 = True                        # @param {"type":"boolean"}
OBS_HEIGHT_1     = 0                           # @param {"type":"number"}
TEMPERATURE_1    = 20                          # @param {"type":"number"}
TEMP_GRADIENT_1  = -0.01                       # @param {"type":"number"}
PRESSURE_1       = 101                         # @param {"type":"number"}
MEASURED_ALT_1   = "92:46"                     # @param {"type":"string"}

LIMB_CORR_1 = int (LIMB_CORR_1)

# SIGHT # 2

In [5]:
# SIGHT NR 2.

USE_2            = True                        # @param {"type":"boolean"}
OBJECT_NAME_2    = "Sun"                       # @param {"type":"string"}
SET_TIME_2       = "2024-06-29 12:51:00+00:00" # @param {"type":"string"}
LIMB_CORR_2      = 0                           # @param [-1,0,1]
INDEX_ERROR_2    = 0                           # @param {"type":"number"}
ARTIFICIAL_HOR_2 = True                        # @param {"type":"boolean"}
OBS_HEIGHT_2     = 0                           # @param {"type":"number"}
TEMPERATURE_2    = 20                          # @param {"type":"number"}
TEMP_GRADIENT_2  = -0.01                       # @param {"type":"number"}
PRESSURE_2       = 101                         # @param {"type":"number"}
MEASURED_ALT_2   = "98:36"                     # @param {"type":"string"}

LIMB_CORR_2 = int (LIMB_CORR_2)

# SIGHT # 3

In [6]:
# SIGHT NR 3.

USE_3            = True                        # @param {"type":"boolean"}
OBJECT_NAME_3    = "Sun"                       # @param {"type":"string"}
SET_TIME_3       = "2024-06-28 15:36:00+00:00" # @param {"type":"string"}
LIMB_CORR_3      = 0                           # @param [-1,0,1]
INDEX_ERROR_3    = 0                           # @param {"type":"number"}
ARTIFICIAL_HOR_3 = True                        # @param {"type":"boolean"}
OBS_HEIGHT_3     = 0                           # @param {"type":"number"}
TEMPERATURE_3    = 20                          # @param {"type":"number"}
TEMP_GRADIENT_3  = -0.01                       # @param {"type":"number"}
PRESSURE_3       = 101                         # @param {"type":"number"}
MEASURED_ALT_3   = "58:40"                     # @param {"type":"string"}

LIMB_CORR_3 = int (LIMB_CORR_3)

# SIGHT REDUCTION

In [7]:
# @title
# SIGHT REDUCTION.

def get_starfixes (drp_pos : LatLonGeodetic) -> SightCollection :
    ''' Returns a list of used star fixes (SightCollection) '''
    a = Sight (   object_name          = OBJECT_NAME_1, \
              set_time                 = SET_TIME_1,              # Time for observation \
              measured_alt             = MEASURED_ALT_1,          # Measured altitude from Sextant \
              limb_correction          = LIMB_CORR_1,             # Semidiameter correction (minutes)  \
              index_error_minutes      = INDEX_ERROR_1,           # Index error correction (minutes) \
              artificial_horizon       = ARTIFICIAL_HOR_1, \
              observer_height          = OBS_HEIGHT_1,            # Observer height (meters) \
              temperature              = TEMPERATURE_1,           # Temperature (degrees celsius)\
              pressure                 = PRESSURE_1,              # Pressure (kPa)
              dt_dh                    = TEMP_GRADIENT_1,
              estimated_position       = THE_POS
              )
    b = Sight (   object_name          = OBJECT_NAME_2, \
              set_time                 = SET_TIME_2,              # Time for observation \
              measured_alt             = MEASURED_ALT_2,          # Measured altitude from Sextant \
              limb_correction          = LIMB_CORR_2,             # Semidiameter correction (minutes) \
              index_error_minutes      = INDEX_ERROR_2,           # Index error correction (minutes) \
              artificial_horizon       = ARTIFICIAL_HOR_2, \
              observer_height          = OBS_HEIGHT_2,            # Observer height (meters) \
              temperature              = TEMPERATURE_2,           # Temperature (degrees celsius) \
              pressure                 = PRESSURE_2,              # Pressure (kPa)
              dt_dh                    = TEMP_GRADIENT_2
              )
    c = Sight (   object_name          = OBJECT_NAME_3, \
              set_time                 = SET_TIME_3,              # Time for observation \
              measured_alt             = MEASURED_ALT_3,          # Measured altitude from Sextant \
              limb_correction          = LIMB_CORR_3,             # Semidiameter correction (minutes) \
              index_error_minutes      = INDEX_ERROR_3,           # Index error correction (minutes) \
              artificial_horizon       = ARTIFICIAL_HOR_3,
              observer_height          = OBS_HEIGHT_3,            # Observer height (meters) \
              temperature              = TEMPERATURE_3,           # Temperature (degress celsius) \
              pressure                 = PRESSURE_3,              # Pressure (kPa)
              dt_dh                    = TEMP_GRADIENT_3
              )
    siteList = list ()
    if USE_1:
        siteList.append (a)
    if USE_2:
        siteList.append (b)
    if USE_3:
        siteList.append (c)

    collection = SightCollection (siteList)
    return collection

collection = intersections = None
try:
    intersections, _, _, collection =\
              SightCollection.get_intersections_conv (return_geodetic=True,
                                                      estimated_position=THE_POS,
                                                      get_starfixes=get_starfixes)
    assert (isinstance(intersections, LatLonGeodetic))
    print ("This is the coordinate of observer's location:")
    print (get_representation(intersections,1))
    print ("")
    print ("This is a link to a map showing the intersections of circles of equal altitude.")
    print ("NOTE: The mapping of these circles is not accurate. Use it mainly for troubleshooting!")
    print ("You may need to Ctrl+Drag the mouse to pan the map")
    print (collection.get_map_developers_string(geodetic=True))
    print ("")
    print ("This is the Google Map intersection position (enter it as an address/location in Google Maps)")
    print (get_google_map_string(LatLonGeodetic(ll=intersections),4))
    print ("")
    int_circle = Circle (intersections, 0.01)
    print ("This is the precise/weighted intersection")
    print (int_circle.get_map_developers_string(include_url_start=True))



except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.")
    if ve.coll_object is not None:
        if isinstance (ve.coll_object, SightCollection):
            print ("Check the circles! " +
                    ve.coll_object.get_map_developers_string(geodetic=True))

the_map = None
if collection is not None and intersections is not None:
    print ("Below you see the intersection using a folium map")            
    the_map = collection.render_folium (intersections)
the_map

This is the coordinate of observer's location:
(Geodetic) (N 57°,38.5′;E 18°,19.0′)

This is a link to a map showing the intersections of circles of equal altitude.
NOTE: The mapping of these circles is not accurate. Use it mainly for troubleshooting!
You may need to Ctrl+Drag the mouse to pan the map
https://www.mapdevelopers.com/draw-circle-tool.php?circles=%5B%5B4844393%2C23.329286%2C55.640583%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B4518696%2C23.317567%2C-11.85025%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B6745149%2C23.367961%2C-53.144%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%5D

This is the Google Map intersection position (enter it as an address/location in Google Maps)
(Geodetic) 57.6424,18.3168

This is the precise/weighted intersection
https://www.mapdevelopers.com/draw-circle-tool.php?circles=%5B%5B1113%2C57.642382%2C18.316812%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%5D
Below you see the intersection using a folium map


© August Linnman, 2025, email: august@linnman.net<br/>
[MIT License](https://github.com/alinnman/celestial-navigation/blob/main/LICENSE)